# Generating a graph from a ".cif" file (MOF structure)

## Prerequisites

### Python Libraries
1. Python (tested on 3.9.1)
2. Numpy (tested on 1.20.2)
3. pycifrw [[conda]](https://anaconda.org/conda-forge/pycifrw) [[pip]](https://pypi.org/project/PyCifRW/) (tested on 4.4.1)
4. *(Optional)* Jupyter / Jupyterlab for running this notebook locally
4. *(Optional)* PyVis for visualization of the graph [[conda]](https://anaconda.org/conda-forge/pyvis) [[pip]](https://pypi.org/project/pyvis/) (tested on 0.1.9)

### Standalone files
All of the listed files below can be found in the `properties` folder
1. Covalent radius of the atoms (included here as **covalent_radius.json**)
2. Electronegativity of the atoms (included here as **electronegativity.json**)

In [1]:
import json
from itertools import product
import numpy as np
import CifFile  # This is the pycifrw module

## Reading the ".cif" file

Some example ".cif" files are included in the `example_cifs` folder. One can generate a graph for a different MOF by changing the `cif_filename` to the path of its coresponding ".cif" file.

In [2]:
# Reading a cif file into a Python-Dictionary-like object
cif_filename = "example_cifs/DB0-m1_o2_o5_f0_pcu_repeat.cif"
mof = CifFile.ReadCif(cif_filename).first_block()

In [3]:
# A simple function for reading a ".json" file into a list of properties
def get_read_json_func(element_list):
    def read_property(path):
        with open(path, 'r') as rf:
            property = json.load(rf)
        return np.array([property[i]  for i in element_list], dtype=float)
    return read_property

## Nodes of the chemical graph

The nodes are atoms in the structure. The following code provides these information about the nodes:
- `node_label`: A label for the atom, which has no physical meaning.
- `node_class`: The type of element of the atom.
- `node_feature_1`: Electronegativity of the atom.
- `node_feature_2`: Covalent radius of the atom.
- `node_feature_3`: Atomic weight of the atom.
- `node_feature_4`: Atomic number (charge) of the atom.
- `node_target`: The atomic partial charge of the atom. This is what we want to use machine learning to acquire.

Note: Not all features may be needed for predicting the target in a machine learning application. In the past experiences, `node_feature_1` (electronegativity) was the most important feature.

In [4]:
# Setting up the function for reading the features
elements = mof["_atom_site_type_symbol"]
read_json = get_read_json_func(elements)

# Putting node information for the graph together
# Each entry is either a Python List or a Numpy Array
node_info = {
    "node_label": mof["_atom_site_label"],
    "node_class": elements,
    "node_feature_1": read_json("properties/electronegativity.json"),
    "node_feature_2": read_json("properties/covalent_radius.json"),
    "node_feature_3": read_json("properties/atomic_weight.json"),
    "node_feature_4": read_json("properties/atomic_number.json"),
    "node_target": np.array(mof["_atom_type_partial_charge"], dtype=float)
}

## Coordinate conversion

Coordinates in the ".cif" files are fractional, the following code generates a [conversion matrix](https://en.wikipedia.org/wiki/Fractional_coordinates) for converting fractional to Cartesian coordinates.

In [5]:
# Reading cell angles
cell_ang = np.deg2rad(np.array([
    mof["_cell_angle_alpha"],
    mof["_cell_angle_beta"],
    mof["_cell_angle_gamma"],
], dtype=float))

# Reading cell lengths
cell_vec = np.array([
    mof["_cell_length_a"],
    mof["_cell_length_b"],
    mof["_cell_length_c"],
], dtype=float)

# Calculate sin/cos of the angles
cell_cos = np.cos(cell_ang)
cell_sin = np.sin(cell_ang)

# Calculate the volume of the cell
cell_vol = np.prod(cell_vec) * \
    np.sqrt(1 - np.sum(cell_cos**2) + 2*np.prod(cell_cos))

# Generate the conversion matrix
frac2cart = np.zeros((3, 3))
frac2cart[0, 0] = cell_vec[0]
frac2cart[0, 1] = cell_vec[1] * cell_cos[2]
frac2cart[0, 2] = cell_vec[2] * cell_cos[1]
frac2cart[1, 1] = cell_vec[1] * cell_sin[2]
frac2cart[1, 2] = cell_vec[2] * \
    (cell_cos[0]-cell_cos[1]*cell_cos[2]) / cell_sin[2]
frac2cart[2, 2] = cell_vol / cell_vec[0] / cell_vec[1] / cell_sin[2]

## Finding the bonds between atoms

Since the structures provided here are periodic, checking atoms across the boundary is necessary for finding out bonded atom pairs. To achieve that. the following code first creates a 3x3x3 supercell, and then finds unique bonded atom pairs within the supercell. Essentially, this is applying the [minimum image convension](https://en.wikipedia.org/wiki/Periodic_boundary_conditions).

In [6]:
# Reading fractional coordinates
frac_xyz = np.array([
    mof["_atom_site_fract_x"],
    mof["_atom_site_fract_y"],
    mof["_atom_site_fract_z"]
], dtype=float).T

# Count the number of atoms in the structure
n_atoms = len(node_info["node_label"])

# Calculates the distance between atoms in vector form in fractional coordinates
frac_diff = frac_xyz.repeat(n_atoms, axis=0) - np.tile(frac_xyz, (n_atoms, 1))

In [7]:
# Initialze a variable for storing all the distances
cart_dist_mirrors = np.zeros((27, n_atoms ** 2))

# Loop over each of the 3x3x3 cell (27 cells in total)
for i, mirror in enumerate(product([-1, 0, 1], repeat=3)):
    # Apply cell adjustments in fractional coordinates
    frac_diff_mirror = (frac_diff + mirror).T
    # Convert to Cartesian coordinates
    cart_diff_mirror = frac2cart @ frac_diff_mirror
    # Calculate the through-space distances from atoms the current mirror image
    # to atoms in the original unit cell
    cart_dist_mirrors[i] = np.linalg.norm(cart_diff_mirror, axis=0)

# Find the shortest distances for each atom pair
cart_dist = cart_dist_mirrors.min(0).reshape(n_atoms, n_atoms)

# Calculate the theoretical bond lengths between each atom pair
cov_r = node_info["node_feature_2"]
cov_r_mat = (cov_r.repeat(n_atoms, axis=0) + np.tile(cov_r, n_atoms)).reshape(n_atoms, n_atoms)

## Edges of the chemical graph

The following code generates a NxN adjacency matrix in the form of a Boolean Numpy Array, where N is the number of nodes in the graph. `True` is assigned where two nodes are connected.

In [8]:
# Tolerance for detecting if two atoms are bonded
bond_tol = 0.25
# Exclude all atom pairs that are not bonded
upper_bound = (cart_dist < (cov_r_mat+bond_tol))
# Exclude all "atom pairs" that has a "zero distance"
lower_bound = (cart_dist > 0)
# Generate the adjacency matrix (with boolean values)
edges = (upper_bound & lower_bound)

### Edge weights

The following line of code generates a matrix that includes the bond distance between two bonded atoms. One can use this as edge weights for the graph. However, it should be noted that **the bond distance is not a qualitative measure of the chemical bond** (i.e. shorter bonds are not always stronger). Therefore using this to train a graph neural network may or may not give better results.

In [9]:
# Using bond distances as edge weights
edge_weights = np.where(edges, cart_dist, 0)

## Visualizing the graph

The following code will generate a graph network and save it as an HTML file. If it does not show properly in the Jupyter Notebook, please open the `test_mof_graph.html` file in a browser.

In [10]:
from pyvis.network import Network

g = Network(width="900px", height="900px", notebook=True)
g.add_nodes(range(n_atoms), label=node_info["node_label"])
g.add_edges(np.argwhere(edges).tolist())
g.show("test_mof_graph.html")